In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
# '\s+' tells pandas to use any spaces as the delimiter instead of ,
original_data = pd.read_csv("../data/delivery.2024.04.16.dat", delimiter='\s+')

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3341820/3529069451.py:2: SyntaxWarning: invalid escape sequence '\s'
  original_data = pd.read_csv("../data/delivery.2024.04.16.dat", delimiter='\s+')


In [3]:
feature_col_names = ['F01', 'F02', 'F03', 'F04',
       'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13']

In [4]:
# Remove gis examples
filtered_data = original_data.drop(np.where(original_data['Tag'] == 'gis')[0])
filtered_data["Tag"].value_counts()

Tag
eq     5837
mis    2043
ex     1540
Name: count, dtype: int64

In [5]:
filtered_data.copy().replace(to_replace='X', value=np.nan)[feature_col_names].astype(float).describe()

,F01,F02,F03,F04,F05,F06,F07,F08,F09,F10,F11,F12,F13
count,4420.000000,3002.000000,5072.000000,3009.000000,5103.000000,3009.000000,5103.000000,1487.000000,2214.000000,2111.000000,1797.000000,1233.000000,2781.000000
mean,-0.117828,0.382648,0.437216,-99.399634,-115.759573,14.756374,13.983525,0.009018,0.023198,0.017186,-0.006277,0.023852,-0.003096
std,0.300832,0.396023,0.388835,22.428991,22.609345,1.286152,1.204947,0.123828,0.167644,0.155244,0.160840,0.288355,0.312698
min,-1.520000,-0.970000,-1.330000,-189.900000,-194.800000,11.640000,10.690000,-0.500000,-0.860000,-0.660000,-0.740000,-0.680000,-0.900000
25%,-0.340000,0.110000,0.200000,-112.900000,-131.300000,13.620000,12.850000,-0.060000,-0.090000,-0.090000,-0.110000,-0.190000,-0.200000
50%,-0.090000,0.370000,0.420000,-100.100000,-117.800000,14.840000,14.210000,0.010000,0.000000,0.000000,-0.020000,-0.040000,-0.070000
75%,0.080000,0.620000,0.660000,-85.600000,-102.100000,15.860000,14.930000,0.080000,0.150000,0.120000,0.080000,0.220000,0.180000
max,2.550000,3.280000,4.220000,-20.200000,-18.500000,17.920000,17.660000,0.470000,0.550000,0.620000,0.660000,1.010000,1.080000


In [11]:
for col in feature_col_names:
    print(col)
    cnt = filtered_data.copy().replace(to_replace='X', value=np.nan)[col].astype(float).unique().shape[0]
    print(f"Entire dataset: {(cnt/filtered_data.shape[0])*100:0.1f}%")
    print(f"With feature: {(cnt/(filtered_data[col] != "X").sum())*100:0.1f}%")
    value_counts = filtered_data.copy().replace(to_replace='X', value=np.nan)[col].astype(float).value_counts().reset_index()['count']
    print(f"Feat1:Feat2: {value_counts[0]/value_counts[1]: 0.2f}")

F01
Entire dataset: 2.1%
With feature: 4.6%
Feat1:Feat2:  1.05
F02
Entire dataset: 2.4%
With feature: 7.7%
Feat1:Feat2:  1.18
F03
Entire dataset: 2.8%
With feature: 5.2%
Feat1:Feat2:  1.07
F04
Entire dataset: 9.8%
With feature: 30.6%
Feat1:Feat2:  1.08
F05
Entire dataset: 11.0%
With feature: 20.3%
Feat1:Feat2:  1.06
F06
Entire dataset: 5.5%
With feature: 17.3%
Feat1:Feat2:  1.06
F07
Entire dataset: 5.6%
With feature: 10.4%
Feat1:Feat2:  1.00
F08
Entire dataset: 0.9%
With feature: 5.6%
Feat1:Feat2:  1.00
F09
Entire dataset: 1.0%
With feature: 4.2%
Feat1:Feat2:  1.03
F10
Entire dataset: 1.0%
With feature: 4.5%
Feat1:Feat2:  1.18
F11
Entire dataset: 1.1%
With feature: 6.0%
Feat1:Feat2:  1.15
F12
Entire dataset: 1.5%
With feature: 11.3%
Feat1:Feat2:  1.04
F13
Entire dataset: 1.8%
With feature: 6.1%
Feat1:Feat2:  1.15


In [8]:
original_data.copy().replace(to_replace='X', value=np.nan)[feature_col_names].astype(float).describe().loc['std']**2

F01      0.096361
F02      0.156913
F03      0.151278
F04    502.488328
F05    510.848576
F06      1.652470
F07      1.449227
F08      0.015308
F09      0.028114
F10      0.024108
F11      0.025805
F12      0.083123
F13      0.097699
Name: std, dtype: float64